In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.collections import QuadMesh
import seaborn as sns

In [ ]:
openknot_data_filepath = './OpenKnotBench_data.v3.1.0.csv'
method_list = ['WT','MPNN-fixbb','gRNAde', 'Eterna'] #'gRNAde2']
out_dir = '/home/afavor/paper_materials/rna_paper/pngs_shape_seq_2025-04-23/'

method_name_to_display_name = {
    'Eterna': 'Eterna',
    'WT': 'WT',
    'MPNN-fixbb': 'NA-MPNN', 
    'gRNAde': 'gRNAde',
    'gRNAde2': 'gRNAde2'
}

In [ ]:
DEFAULT_EXPERIMENTAL_PALETTE = {
    'NA-MPNN':  "#2ECC71",
    'gRNAde':   "#D3D3D3",
    'gRNAde2':  "#808080",
    'WT':       "#4B4BFF",
    'Eterna':   "#A9A9A9", 
}

DEFAULT_BOX_STYLE = {
    'figsize': (40 / 25.4, 40 / 25.4),
    'dpi': 300,
    'title_fontsize': None,
    'axis_title_fontsize': 8,
    'tick_labelsize': 6,
    'palette': DEFAULT_EXPERIMENTAL_PALETTE,
    'title': None,
    'x_label': None,
    'y_label': 'OpenKnot Score',
    'y_max': None,
    'y_min': None,
    'background_color': 'white',
    'show_title': False,
    'show_axis_labels': True,
    'show_yticks': True,
    'show_xticks': False,
    'save_name': None,
    'show_legend': True,
    'legend_title': None,
    'legend_loc': None,
    'legend_ncol': 1,
    'legend_title_fontsize': None,
    'legend_fontsize': 6,
    # legend frame/handle controls
    'legend_frameon': False,
    'legend_mode': 'expand',
    'legend_handlelength': 1.5,
    'legend_handleheight': 0.3,
    'legend_labelspacing': 0.3,
    'legend_borderpad': None,
    'legend_markerscale': None,
    'legend_handletextpad': 0.2,
    'legend_handle_linewidth': 0.5,
    'legend_figsize': (16 / 25.4, 14 / 25.4),
    # line width controls for boxplot elements and axis appearance
    'box_linewidth': 0.5,
    'whisker_linewidth': 0.5,
    'cap_linewidth': 0.5,
    'median_linewidth': 0.5,
    'median_linecolor': None,        # color for median line
    'spine_linewidth': 0.5,
    'tick_width': 0.5,
    'hide_top_right_spines': True,
    # outlier marker controls
    'flier_markersize': 2.5,
    'flier_linewidth': 0.5,
    # separate legend saving controls
    'save_legend_separately': True,
    'legend_save_suffix': '_legend',
    'legend_fill_figure': True
}

DEFAULT_HEATMAP_STYLE = {
    'figsize': (125 / 25.4, 70 / 25.4),
    'dpi': 300,
    'title_fontsize': None,
    'axis_title_fontsize': 8,
    'tick_labelsize': 6,
    'cmap': 'YlGnBu',
    'title': None,
    'x_label': None,
    'y_label': None,
    'vmin': 0.0,
    'vmax': 0.3,
    'background_color': 'white',
    'show_title': False,
    'show_axis_labels': True,
    'show_ticks': True,
    'save_name': None,
    # axis/spine/tick controls
    'spine_linewidth': 0.5,
    'tick_width': 0.5,
    'hide_top_right_spines': True
}

DEFAULT_SCATTER_STYLE = {
    's': 10,
    'palette': DEFAULT_EXPERIMENTAL_PALETTE,
    'title_fontsize': None,
    'axis_title_fontsize': 8,
    'tick_labelsize': 6,
    'title': None,
    'x_label': 'OpenKnot Score',
    'y_label': None,
    'background_color': 'white',
    'show_title': False,
    'show_axis_labels': True,
    'show_yticks': False,
    'show_xticks': True,
    'save_name': None,
    'show_legend': True,
    'legend_loc': None,
    'legend_ncol': 1,
    'legend_title_fontsize': None,
    'legend_fontsize': 6,
    # axis/spine/tick controls
    'spine_linewidth': 0.5,
    'tick_width': 0.5,
    'hide_top_right_spines': True,
    # separate legend saving controls
    'save_legend_separately': True,
    'legend_save_suffix': '_legend',
    # legend controls
    'legend_frameon': False,
    'legend_mode': 'expand',
    'legend_handlelength': 1.5,
    'legend_handleheight': 0.3,
    'legend_labelspacing': 0.3,
    'legend_borderpad': None,
    'legend_markerscale': None,
    'legend_handletextpad': 0.2,
    'legend_handle_linewidth': 0.5,
    'legend_fill_figure': True,
    'legend_figsize': (18 / 25.4, 9 / 25.4),
}

In [ ]:
font_path = "./ARIAL.TTF"
matplotlib.font_manager.fontManager.addfont(font_path)

# 2) Tell Matplotlib to load it
plt.rcParams['font.family'] = "Arial"

In [ ]:
openknot_data_dict = {
    # Openknot round 5 and 6 data:
    'W01':
        {
        'str':'(.((......))..).((((((((((.(([[[[[))))))))))))(((((((((..(....).))).)))))).....]]]]]....',
        'seq':'GUUUUUAAACGGGUUUGCGGUGUAAGUGCAGCCCGUCUUACACCGUGCGGCACAGGCACUAGUACUGAUGUCGUAUACAGGGCUUUUG',
        },
    'W02':
        {
        'str':'((((....((((((((((((......(((((((...[[[[[...))))))).....((....))))).))))))))).]]]]].)))).',
        'seq':'GGAGAGUAGAUGAUUCGCGUUAAGUGUGUGUGAAUGGGAUGUCGUCACACAACGAAGCGAGAGCGCGGUGAAUCAUUGCAUCCGCUCCA',
        },
    'W03':
        {
        'str':'(((((.(((......(((((((((..(..{[[[[[[[))))))))))...)))...]]]]]}.]]..)))))..',
        'seq':'CGCGGAAACAAUGAUGAAUGGGUUUAAAUUGGGCACUUGACUCAUUUUGAGUUAGUAGUGCAACCGACCGUGCU',
        },
    'W04':
        {
        'str':'(((((((((((.((((((.[((((....)))))])))))[))((((..].))))((..[[[[[.))))))))))).]]]]]...',
        'seq':'CCUCCCGGGAGAGCCGCUAAGGGGGAAACUCUAUGCGGUACUGCCUGAUAGGGUGCUUGCGAGUGCCCCGGGAGGUCUCGUAGA',
        },
    'W05':
        {
        'str':'.(((((((((((((((((((..[[[[[[.)))))(((....)))(((....)))))))))))))))))((((((..]]]]]].)))))).',
        'seq':'CCUGGGAACCUCCUGUGCUGAAGCGCGCACGGCAACCGAAAGGUGCGGAAACGCUGGGGGGUUCCUGGAUGCUGAAGCGCGCACGGCAUU',
        },
    'W06':
        {
        'str':'((((......[[[[[[...))))(((((((((........)))))))))....((((((((((........))))))).)))..]]]]]]..........',
        'seq':'GAGCAACUUAGGAUUUUAGGCUCCCCGGCGUGUCUCGAACCAUGCCGGGCCAAACCCAUAGGGCUGGCGGUCCCUGUGCGGUCAAAAUUCAUCCGCCGGA',
        },
    'W07':
        {
        'str':'.[[[[[..........((((((((....))))))))(((((((((((..]]]]]...........)))))))))))',
        'seq':'GGAAGGUUUUUCUUUUCCUGAGGCGAAAGUCUCAGGUUUUGCUUUUUGGCCUUUCUUAAAAAAAAAAAAAGCAAAA',
        },
    'W08':
        {
        'str':'[[[[[......((((((((((((]]]]].....).).)))))))))).',
        'seq':'GGUUUCUUUUUAGUGAUUUUUCCAAACCCCUUUGUGCAAAAAUCAUUA',
        },
    'W09':
        {
        'str':'.(((((.......(((((((((............{{{{{{{.......))))))))).....((((..}}}}}}}.)))))))))...............',
        'seq':'ACUGCGACAGAAAAGAAGCGUGAAACAAAAUGAAUCUCGGCAAAGCUACGCGCUUCUACAUACCAGCAGCCGAGAACUGGCGCAGAUUACAAACGCAGAC',
        },
    'W10':
        {
        'str':'(((((((..............(((((......{{{{{{.....))))).......((((((.......}}}}}}.....))))))....)))))))....',
        'seq':'GAGCCCGUUGCAUUUUAUUUUCGAGCUUUCUUGCCGCCUUCUUGCUCGUAUUGCUGGACCGUUUCUUUGGCGGCUUCUUCGGUCCUAUUCGGGCUCGAGA',
        },
    'W11':
        {
        'str':'((.(({{{{{{{.....((((((.......)))).)))).))......(((((((((((((((((...}}}}}}}))))))))...))))))))).....',
        'seq':'GCUGGCCUGCGCACGAUCUGACUGCCGUGAAGUCUAGCCUGCGUCAGAACUAGUCUCCAUCAAUCAAAGCGCAGGGAUUGAUGUACGAGACUAGUACACA',
        },
    'W12':
        {
        'str':'...(((((.{{{{{{.)))))....(((((((.......}}}}}}...........(((.((((........)))).)))((....))....))))))).',
        'seq':'CUACCUUUCCAUGCUAAAAGGUUCAAUGAUUGGUUAGAAAGCAUGUCAAAUAAUGUCGUUCGAUCAAAAGGUAUCGUAUGGUUAGCACAAUACAAUCAUG',
        },
    'W13':
        {
        'str':'.((((((((((.((((((..{{{{{.....(((....)))............))))))...........(((.......)))}}}}})))))))..))).',
        'seq':'UGGCCUACCAUACUAUCCGUUAUAUAAAUAGGCAAAUGCUCUUUUCCAUUAUGGAUAGCAAUAGUAUGACCGAUCAUUGUGGGUAUAAUGGUAGAUGCCC',
        },
    'W14':
        {
        'str':'(((((....((((....))))(((..((((((.[[[[[.))).))).)))..(((((....)))))))))).((((....)))).......]]]]].',
        'seq':'GGAUCACGAGGGGGAGACCCCGGCAACCUGGGACGGACACCCAAGGUGCUCACACCGGAGACGGUGGAUCCGGCCCGAGAGGGCAACGAAGGUCCGA',
        },
    'W15':
        {
        'str':'((((((((((((((((((((((...[[[[.))))))).).)))).))))).((..........)).....)))))..........]]]]',
        'seq':'GGGGGCCAGAUGUCAUGUCUCUCAAGCCUAGGAGACACUAGACACUCUGGACUAUCGGUUAGAGGAAACCCCCCCAAAAAUGUAUAGGC',
        },
    'W16':
        {
        'str':'(((((((...[[[[[[(((.[[.....))))))))))]]....((((..........)))).....]]]]]]',
        'seq':'GGCCGGCAUGGUCCCAGCCUCCUCGCUGGCGCCGGCUGGGCAACACCAUUGCACUCCGGUGGCGAAUGGGAC',
        },
    'W17':
        {
        'str':'[[[[[[...((((((((((.......))).]]]]]]..(((((..........)))))....)))))))',
        'seq':'GGGGGCCACAGCAGAAGCGUUCACGUCGCAGCCCCUGUCAGCCAUUGCACUCCGGCUGCGAAUUCUGCU',
        },
# openknot 7a puzzles:
    'P01':
        {'other_name':'Thermotoga_petrophila_fluoride_riboswitch',
        'seq':'GGGCGAUGAGGCCCGCCCAAACUGCCCUGAAAAGGGCUGAUGGCCUCUACUG',
        'str':'.[[[[{.((((((]]]]......(((((....)))))}.[.))))))]....',
        },
    'P02':
        {'other_name':'ZTP_switch',
        'seq':'UAUCAGUUAUAUGACUGACGGAACGUGGAAUUAACCACAUGAAGUAUAACGAUGACAAUGCCGACCGUCUGGGCG',
        'str':'.....(((((((..(.[[[[[...((((......))))..)..))))))).........(((..]]]]]..))).',
        },
    'P03':
        {'other_name':'pfl_switch',
        'seq':'GGGAUACAGGACUGGCGGAUUAGUGGGAAACCACGUGGACUGUAUCCGAAAAAAAGCCGACCGCCUGGGCAUC',
        'str':'.((((((((..(.[[[[[....((((....))))..)..))))))))........(((..]]]]]..)))...',
        },
    'P04':
        {'other_name':'sgRNA',
        'seq':'GGAAAUUAGGUGCGCUUGGCGUUUUAGUCCCUGAAAAGGGACUAAAAUAAAGAGUUUGCGGGACUCUGCGGGGUUACAAUCCCCUAAAACCGC',
        'str':'....................((((((((((((....))))))))))))..((((...[[[[..))))..((((.......)))).....]]]]',
        },
    'P05':
        {'other_name':'PreQ1-II_switch',
        'seq':'GCUUGGUGCUUAGCUUCUUUCACCAAGCAUAUUACACGCGGAUAACCGCCAAAGGAGAA',
        'str':'((((((((.....[[[[[[.)))))))).........((((....))))..]]]]]]..',
        },
    'P06':
        {'other_name':'Grapevine_leafroll-associated_virus_-_2',
        'seq':'ACGCCAAAAUCCAAUUAAAGUUUGGGACCUAGGCGGGCCUCUUACGAGGCUAACUUAUCGACAAUAAGUUAGGUC',
        'str':'.((((.....((((.......))))[[[[[[))))((((((....))))))..................]]]]]]',
        },
    'P07':
        {'other_name':'Turdivirus_3',
        'seq':'UCCGUCCAAGCCGCGGACGUUAAACUGUGGCACGGUUGUGCCUAGGUGCAACCCUGCUACUAAUGGCGGUACCCCUGCCC',
        'str':'..(((((..[[[[[))))).......((((((.((((((((....)))))))).)))))).....]]]]]..........',
        },
    'P08':
        {'other_name':'Poa_semilatent_virus',
        'seq':'AUUGGUAUGUAAGCUACUUCUUCCAGUAGCUGCGUCAUAACAUCAAGGUUAUGCAUACUGAGCCGAAGCUCAGCUUCGGUCCUCCAAGGAAGACCA',
        'str':'.((((((((..((((((..[[[[[.))))))....((((((......)))))))))))))).((((((.....))))))........]]]]]....',
        },
    'P09':
        {'other_name':'E._coli',
        'seq':'GCGUAAAUGUCGACUUGGAGGUUGUGCCCUUGAGGCGUGGCUUCCGGAGCUAACGCGUUAAGUCGACC',
        'str':'..((....[[[[[[..((((((((((((.....))))))))))))...[[..))]].....]]]]]].',
        },
    'P10':
        {'other_name':'Diplonema_papillatum',
        'seq':'GAGGGACAAGAAUCUGACCUGCACCUCCUCGUGGUGUCCCUCGGAAACGUGCUCAACGCGCGGCCGACGCAGGCAG',
        'str':'(((((((......[[[.[[[[[(((.[[....))))))))))]]...(((((.....)))))......]]]]]]]]',
        },
    'P11':
        {'other_name':'PN.v282',
        'seq':'CUGCCGACGCCCUGUGUUGAGGCGUGGUGUCGCCUCGAGUCAGUCGUUGAGAUCCGAAGAAGGGUGUACUUCUGACUGAUCGAGCAGAAGUGAGUCGAGG',
        'str':'(((((((((((.(((......))).)))))))[[[[[[((((((((..(((((((......))))...))).))))))))...)))).......]]]]]]',
        },
    'P12':
        {'other_name':'mod_of_f67',
        'seq':'GAAAAGAGCGUAAUCGCGUCUUGACGUGAGUGAGGCUAGACUGUAGACUAUCCCAAGUGGGAUAGCAUAACUCACAAUACGCUGAUCUACAGUCAAAAGA',
        'str':'......((((((.((((((....))))))(((((....[[[[[[[[[(((((((....))))))).....)))))..))))))..]]]]]]]]]......',
        },
    'P13':
        {'other_name':'UC2414',
        'seq':'GGAAGACAACUCGCGGUCUUCCAAGUACUCGAAGGAAGGCGAGACGCACGAGUGCAGCGUCAAAUGACACCACGUCAUCCAACGCGAGAAGGUGUCAGAA',
        'str':'(((((((..[[[[[[)))))))..((..((....))..))..(((((.........)))))...(((((((...........]]]]]]..)))))))...',
        },
    'P14':
        {'other_name':'SV_j116',
        'seq':'GAGGUGAUAAUCUGAUAGGAGGCUGAGAAGUCUGAAGGUAAAUUAUCGUCUAAUUAAAUCUGGUUUAAGUACCUAUCAGAUCUUAGACUAGAUGAAGAUA',
        'str':'.(((((((((([[[[[[[[(((((....)))))........))))))))))......((((((((((((..]]]]]]]]..)))))))))))).......',
        },
    'P15':
        {'other_name':'UC2458',
        'seq':'GUGGCACGUGUGCAGCCACGAAGGUUGGUAACACCGGAUCAACCGAUGUACACGACGUCUCGGAAUGCCAUGUUGAGACGUAUGGUAUUCCAACCGGUGA',
        'str':'(((((.[[[[[[[[)))))...(((((((..{{{{{{)))))))..]]]]]]]].......(((((((((((........)))))))))))..}}}}}}.',
        },
    'P16':
        {'other_name':'AK_PK100-3',
        'seq':'AGUGUGCCCCGGACCUCGCACACAAAGGUUGGGAGGUGGGGACCAACCAAAAGCAUCGUCCCCGGCACCGAUGCAUAGCCUGGGUGCCCCGGGCCAGGCA',
        'str':'.(((((([[[[[[[[[[))))))...((((((]]]]][[[[[))))))....((((((]]]]][[[[[))))))...((((((]]]]]]]]]])))))).',
        },
    'P17':
        {'other_name':'ZG21',
        'seq':'GGCCAGACAGCGUAACCGUGGCCCAAAAGGGGCGACGAGUACGGUAUCGUCGCGGAACUAUACCGGAUGGUGAACCGCCACGGACGCAUCACCAUUUGCC',
        'str':'(((......((((..(((((((((.....)))((((((...[[[[.))))))(((......]]]][[[[[[[..)))))))))))))..]]]]]]].)))',
        },
    'P18':
        {'other_name':'Terminal_3',
        'seq':'AGGUAAGUUGGAAGGUGUGAGGUGUUUCAUCAUCAUCUCACACCAAAGUACUUGCGAGAAAAAGUACUAACAACUUGCCAGCAAUGAUGAUUUCUCGACU',
        'str':'.(((((((((...(((((((((((..[[[[[[.)))))))))))..(((((((.{{{{{{.)))))))..))))))))).....]]]]]].}}}}}}...',
        },
    'P19':
        {'other_name':'SV_r7_100_a7',
        'seq':'GCUUCCGAAUGAUGACUCAAAUAGGUAAGUUGGAAGGUGUGAGGAGUCAUCAUCUCACACCUAAGUACUUGUUCGGAGGCCAAGUACUACCAACUUGCCA',
        'str':'((((((((((((((((((.....[[[[[[[[[[.[[[[[[[[[))))))))))]]]]]]]]].[[[[[[[[.))))))))]]]]]]]].]]]]]]]]]].',
        },
    'P20':
        {'other_name':'Kissing_multiloops',
        'seq':'GGACGACAGUCUGCUUGCAGACCUCCGAAAGGAGAUGGCCAGUCGUCCUGGGUCAUCAGGCCAGUCGUUCGCGACGGUAGCGUGAGCUACCAGAUGACCU',
        'str':'(((((((.(((((....)))))((((....)))).[[[[[.))))))).((((((((.]]]]]((((....))))((((((....)))))).))))))))',
        },
# openknot 7b puzzles:
    'Q01':
        {
        'other_name' : 'CRISPR_Guide_7YOJ',
        'str':'.((((.(...(.(((((..(.(.((....((((...(((.....)))...))))......)).).)..((((....)))).....)))))...)..(((.[[[[..)))).)))).(((((((((..((.....))..)))))))))...]]]]....................',
        'seq':'GUCUGCCGAAGACGCCGCACGGAGCCUGGGCCGGAAUCGUAGAUCGAACGCGGCAUCGAAGCCCUGCAGCCCUUCGGGGCCAAGGCGGCGCAGCAAGCCUCUUUCAGGCGGCAGAGUCCUUUAGAGUGUGAGAGACACUCUAAAGGAAUGAAAGAGGGCGACACCCUGGUGAAC'
        },
    'Q02':
        {
        'other_name' : 'Nanobracelet_7JRT',
        'str':'(((((((((((.......((((((.....((((((..[[[[[[.)))))))))))))))))))))))((((((((((.]]]]]].((((((.....((((((.........)))))))))))).))))))))))',
        'seq':'GGAUAUUCGACGGAGGCACCCAGGAACUACCGUUGAAGCUCGCACGACGGCCUGGGGUCGAGUAUCCCGGUAUUUGUCGCGAGCACUGAGGAACUACUGCUGAAGCCUCCACGGCAGCCUCAGGACAAGUACCG'
        },
    'Q03':
        {
        'other_name' : 'GLMS_ribozyme_3G9C',
        'str':'(((((..........))))).......(((...[[[[[[..)))..............]]]]]]((((((.[[[[[[))))))...(((((((((.....(((((((....)))))))......))))))))).]]]]]]',
        'seq':'GCACCAUUGCACUCCGGUGCCAGUUGACGAGGUGGGGUUUAUCGAGAUUUCGGCGGAUGACUCCCGGUUGUUCAUCACAACCGCAAGCUUUUACUUAAAUCAUUAAGGUGACUUAGUGGACAAAGGUGAAAGUGUGAUGA'
        },
    'Q04':
        {
        'other_name' : 'Tuberculosis_T-box_6UFG',
        'str':'(((......((((......))))...)))(((((......((((.....))))........)))))(((...[[[))).....]]].........(..(((((((....)))))))..[[[[[[......)(((((((.(((((....))))))))))))]]]]]]',
        'seq':'GGCAUCGAUCCGGCGAUCACCGGGGAGCCUUCGGAAGAACGGCCGGUUAGGCCCAGUAGAACCGAACGGGUUGGCCCGUCACAGCCUCAAGUCGAGCGGCCGCGCGAAAGCGUGGCAAGCGGGGUGGCACCGCGGCGUUCGCGCGAAAGCGUGGCGUCGUCCCCGC'
        },
    'Q05':
        {
        'other_name' : 'Ligase_ribozyme_3HHN',
        'str':'((((((((...[[[[[[.))))))))...............((((((...).[.[[[[[[[(((((((..........)))))))..[[[[.)))))]]]]((.((((......)))).))]]]]]]]].]]]]]].',
        'seq':'UCCAGUAGGAACACUAUACUACUGGAUAAUCAAAGACAAAUCUGCCCGAAGGGCUUGAGAACAUACCCAUUGCACUCCGGGUAUGCAGAGGUGGCAGCCUCCGGUGGGUUAAAACCCAACGUUCUCAACAAUAGUGA'
        },
    'Q06':
        {
        'other_name' : 'Taura_IRES_5JUP',
        'str':'...(((((((..............(((((........[[[[[..........))))).........)))))))((((.(..........)..[[[[[))))(.((.....)).)..]]]]].....]]]]]..(((((.(((((((.[[[[[)))).))).((((((....)))))))))))........]]]]]......',
        'seq':'AAACUCCAUGUAUUGGUUACCCAUCUGCAUCGAAAACUCUCCGAACACUAGGUGCAGUAAGGCUUUCAUGGAGUGGUUUGCUAUUUAGCGUACGUGUACCAUAGGCAGCCCCAAAAACACGUGUGAGGAGAAAGUCCCAGUCACUUUGGGCAAAGUAGACAGCCGCGCUUGCGUGGUGGGACUUAAUUAAUGCCUGCUAAC'
        },
    'Q07':
        {
        'other_name' : 'Broad_bean_mottle_virus_PKB135',
        'str':'..(((((((..((((((..[[[[[[)))))).....(((((.(((.((.....)).)))....))))))))))))(((((((((....))))))))).........]]]]]]....',
        'seq':'AAGAUAUCUGUACCGUCUUCUCCCCGACGGUUUCAAUCCAUAGCUUCACGUUGUGUAGCGUAGGUGGAAGGUAUCGUGUGGUGUAUAAACACCACAUAGUCUCUUAGGGGAGACCA'
        },
    'Q08':
        {
        'other_name' : 'Rous_sarcoma_virus_PKB174',
        'str':'........((((((((((((((...........[[[[[[[[((((((..(((((......)))))))))))((((((....))))))......))))))))).)))))...........]]]]]]]].',
        'seq':'AAAUUUAUAGGGAGGGCCACUGUUCUCACUGUUGCGCUACAUCUGGCUAUUCCGCUCAAAUGGAAGCCAGACCACACGCCUGUGUGGAUUGACCAGUGGCCCCUCCCUGAAGGUAAACUUGUAGCGCU'
        },
    'Q09':
        {
        'other_name' : 'Methylosinus-1_RF03108',
        'str':'.(((.(((...((((((.....((((((....)))).))[[[[[[[[))))))))).)))....(((.(((((((((((.......))))(((((((....)))))))..))))))))))........(((((]]]]]]]].........(((....))).......)))))...',
        'seq':'AGGGUGCGGUUUUCGAGAGAAGCGGGGCGGCUGCCCUCGUCCGCGCGCUCGAACGCUCCUCCGCCGGUCCGGCUUCUCCUUCGUGCGGAGGCCUUUGGAAACAAAGGCGAAAGUCGGCCGAAUGUCCGAGGGUCGCGCGGGGAUAGACUACCCGAGAGGGGAAAACAACCCUGGC'
        },
    'Q10':
        {
        'other_name' : 'SCARNA2_URS00008E39F0_9606',
        'str':'.(((....(((((((((((.((((((((((((((((..(((((((([[[[[[.............))))((((((((....)))))))).....((((..............)))).(((((.(((..........)))..)))))..)))).....)))))))......)))))))))..)))]]]]]].))))))))...))).',
        'seq':'ACACAGGUGUGGGCUUACCUGACCAUGGCUGUCAAUAAGGAGGGGGGAGGUGUACUGGUAAUGAUCCCCACAGCUUGUUGGUAAGCUGUUACAGGAGCUUUAGGCAACACUUGCUCUGGCCAUGCCCUUCAGUCCUGGCCCUGGCUAUCUCUGUUCUGUUGGCGCAUGAUGCCAUGGUUUUGGUCACCUCAAGGCCUGCUUAGUGC'
        },
    'Q11':
        {
        'other_name' : 'SV_g',
        'str':'..((((((((((([[[[[[[[[.((((((((((((........(((((((((((((...........)))))))))))))........))))))))))))........))))))))))).((((((((((]]]]]]]]](((((((((.((............((((((((((............))))))))))............)).)))))))))...........))))))))))',
        'seq':'GACACAUCUCACACUCCUCCUGCCUCACUCUGUUUAAAGGAGACGAGGGUGCAAACCUCAAUUCUUAGUUUGUACUUUCGCAGAAGAAAAACGGAGUGAGAAUUUAUAUGUGAGAUGUGACUAGCUUCCUUAGGAGGGGUAUGCUCACAUUUUUCUUCUGCGGAAGAUGUUUCAUAAGAAUUGAGGAAACAUCUUAUACUUUUAUAAAGAGUGAGUAUAGAGGAAGAGAAAGGAAGCUAG'
        },
    'Q12':
        {
        'other_name' : 'SV_i',
        'str':'...((((((((((............[[[[[[[[))))))))))....((((((((((]]]]]]]].[[[[[[[[[[[[))))))))))....((((((((((]]]]]]]]]]]][[[[[[[[[))))))))))....((((((((((]]]]]]]]]..[[[[[[[[[))))))))))....((((((((((]]]]]]]]]............))))))))))....((((....))))..',
        'seq':'GGAAGUCUGUAGAAUUACUGUUAAAUUCCCUUAUCUGCAGACUAUGAAGUAGAGAUCUAAGGGAAUAAGGUAGGAAUUGGUCUCUACUAUGAGGGUUCUCUGAAUUCCUACCUUAAACGCGUACAGAGAACCCAGUAACAUACGUCGUACGCGUUUAAAAAUCUCAUCGACGUAUGUAGUAAGUUAGUGUGAUGAGAUUUUAAGGAGAGGAACACACUAGCUAAAUGGUUGAGAAACCUA'
        },
    'Q13':
        {
        'other_name' : 'SV_h3',
        'str':'.((((((.......(......).........(((((((((((((.[[[[[[[[[[[[[[[[.............[[[[[[............(((((((((((((((((...(((((...........)))))......))))))))))))))))))))))))))))))..]]].]]]..]]]]]]]]]]]]]]]].)))))).',
        'seq':'AAUCACUAAGGUUUAGGGAAAUGAAGAUGGGUGACUCAAUUCUUAGAAUAUCGUAACGAUGAAGGAGUAUAAUGUGAGGAUUUUCAUGUAAGAUGUCACUAGCUUCCUUCUUACAUGCAAAGUUCUCACAUGUUCCUUCAAGGAAGCUAGUGACAUAAGAAUUGAGUCAAUUCCAUCAACUAUCGUUACGAUAUUCAAGUGAUG'
        },
    'Q14':
        {
        'other_name' : 'SV_c',
        'str':'.((((((((((((((((.......))))))))((((((((((((((.......))))))))((((((((....(((((((((((....((......((((((([[[[[[[[[)))))))..))....)))))))))))....))......(((((((]]]]]]]]])))))))))))))((((((((.......))))))))))))))((((((((.......)))))))))))))))).',
        'seq':'GUAGUCUGUGUCGUACACCGAGCCUGUACGACCCUACUCGUAGUACUGUAUGUGUGCUACGCGUGAGGUAAGGAUGUGACUGAUUCUUACGCAUGCGUAGGUUUCCAGUCUAAAUCUACAAGUUUAUAUCAGUCACAUAUGAACGCAUGCCAAUAGGUAGACUGGACUUAUUGCUCACGGGAGCUCAGGCUCGGUGAGCUUCAGUAGGACCGGUACACAUACAGUACCGGUACAGACUAG'
        },
    'Q15':
        {
        'other_name' : 'SV_f',
        'str':'.((((((((((((((((((((.........))))))))))).(((((((..(((((((((.........)))))))))(((((((((((((((((((((.........))))))))))).....((([[[[[[[[[[[[[[[[...)))...))))))))))(((((((((.........)))))))))..)))))))..(((((...]]]]]]]]]]]]]]]]))))).))))))))).',
        'seq':'GAGAGUCUGUGUUGUACACUGAUUGCUAACCAGUGUACAACAUAGUACUAGAUGUGACAGCUACGACACCUGUCACGUGAGACUGCUCGUUAUGCAUUGGUUAGCAAACAAUGCGUAACAACAUGUUGCAAGUCCAACUUCCAUGCGACAGAGAGUAGUCUCCGGAGUUGCGUGUUGUAAGCAGUUUCGAUAGUACUAUGCUAGUACCUGGAAGUUGGACUUGUACUAGCACAGACUCUA'
        },
    'Q16':
        {
        'other_name' : 'AK_PK240-3',
        'str':'.((((.(((((.(((((.....))))))))))......((((((((((.....)))).))))))(((((((.(((.....))))))))))...)))).....((((((((((((((.....)))))..((((((.....))))))(((.((((((.[[[[[[[[.)))))))))..)))))))))...(((((((((((..(((((.]]]]]]]].))))).......))))))))))).',
        'seq':'AGGACAGACUGACGGUCGAUUAGACUGCAGUCAAUUAAGAUCAGCUACGAUUAGUAGACUGGUCGAUCUUCAGACGAUUAGUCGAGGAUCAUAGUCCAAUAAGAUUAUAUGCUGGCGAUUAGUCAGAAGAUGAGGAUUACUCGUCGAGACAUAUCAGGAUAGGGAGAUAUGCUCAACAUGUAAUCAAAGAUAAUGUUCCAAGAUUCACCCUAUCCAGAAUCAAUAUAAGGAAUAUUAUCA'
        },
    'Q17':
        {
        'other_name' : 'pknot240_17',
        'str':'....(((((((((....[[[[...))))))))).(((((((((.......)))))))))..((((((.....))))))...((((((.....[[[[..))))))..((((((........))))))...(((((........)))))..........((((......))))...(((((........)))))..(((((.]]]]...))))).(((((((....]]]].)))))))....',
        'seq':'AAGAGUAUAUAGCACCAGGCCAAGGCUAUAUACACACAUGUGUACGACAAACACAUGUGACGUAUGCAGUAGGCAUACCGGGGCUCGAUGAGAGGGCGCGAGCCAGGCCACGAAGCCAGACGUGGCAGAGUGCCAUAAUACCGGCACAGGAAACCAAGCCCAUAUAGGGGCCAUGGUCCAAGAAUAUGGACCCAGCUACACCCUAGAGUAGCAGGACACAAUGUGGCCAUGUGUCCGAUA'
        },
    'Q18':
        {
        'other_name' : 'Astros-Eli-mod',
        'str':'((((((((...(((((((((((.[[[[[[[..)).)..)).)))))).......(.(((((.((((((((((.[[[[.))))))))))...))))).)....)))))))).((((((.((((((((((..(((..((]]]]...)).)))((((.(((((.((((.(((((((....))))))).((((...(...]]]]]]])))))..))))))))))))))))))))))))))))).',
        'seq':'GGCAAAGGAUGAAGAGGACCGGAACUGACCAGCCAGCUGUCCCUCUUACCUAAAGACUUAAACCAAUGCCCUAGUGAGGGGGCAUUGGGCAUUAAGCCGUGACCUUUGCUAGAGAAUUAAAUGAGAUAAAAGAGGCCUCACGCAGGAUCUGGCAUAGAGGAGGUGAUCAGCAAAUGUUUGUUGAAAAGGUUUGACAGGUCAGUCCCUUCCCACCCCUCUUGCUUGUCUUAUUUAUUCUCA'
        },
    'Q19':
        {
        'other_name' : 'SV_r7_240_4',
        'str':'((((((((((((.(((((((((.[[[[[..))))))))).(((((((((.((((((((((((((((.((((((((((((......)))))))))))).(((((((((((..]]]]].))))))))))).))))))))).((((((((((..[[[[[[.....))))))))))...))))))).(((((((((((.]]]]]].....))))))))))).))))))))).))))))))))))',
        'seq':'UCUGUGGUCUGUAGGUAGAGCGAAGGAUAACGUUCUACCACCUCGACUCAGGUUAGAGGCAGCGGGAGGUUUGGGGGAAACAGCAUUUCCUCAAGCCAGGUCUUGUGUUUUAUCCUUAACACGAGGCCACCUGCUGCCACGUGGCUCGAAAUCCGGUGACAAUCGAGUUACGACGUCUGAUCACACAUCGAGCAAACCGGAAAAGAUGUUCGAUGUGAGAGUUGAGGAACAGACUACAGA'
        },
    'Q20':
        {
        'other_name' : 'Crazy_Legs_study_7',
        'str':'.......((......(((((..((..[[[[[[[........))..(...((((((...(((((((.(.((......))).....))).))))....))))))...).....]]]]]]]))))).......[[[[[[...((((((.(.....(..[[[[[[.)........[).))))))....{{{{{{.{....]........]]]]]]..)).....}.}}}}}}...]]]]]]...',
        'seq':'AAGCGGUUCUUGAAAGCGCGAUGGGAUUGCUAAACAUGUUGCCAUGAGACUAGUCGAUGCGCAGACAUUGUUAAGUCAUGAAAAUCUAGCGCGAAUGAUUAGGAUUAAACGUUAGCGACGCGCAAAACAGUUUUCGUAAUGUACGGCGAAUCGAUUAUACGUCGAAUAUCGAGCUGUGCAUAAAGUUUGGACAGUAUCAAAACGAUGUAUAAAGAACUAAGUUCGAGCUGACGAAAAGUC'
        },
    'miscellaneous':
        {
        'other_name' : 'Pseudoknot_240',
        'str':'.....((((((.....))))))....................................................................................................................................................................................................................................................((((((((....)))))))).....................',
        'seq':'GGGAACGACUCGAGUAGAGUCGAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAUUCGAAAAAAAAAAAAGAAACAACAACAACAAC'
        },
    }

# Helper Functions for Plotting

In [ ]:
def _save_legend_from_axes(ax, parent_save_name, style_dict):
    """Save legend from ax as a separate figure and remove it from parent."""
    if not parent_save_name:
        return None

    lg = ax.get_legend()
    if lg is None:
        return None

    handles, labels = ax.get_legend_handles_labels()

    try:
        lg.remove()
    except Exception:
        pass

    dpi = style_dict.get('dpi', 300)
    suffix = style_dict.get('legend_save_suffix', '_legend')
    root, ext = os.path.splitext(parent_save_name)
    if not ext:
        ext = '.svg'
    save_path = f"{root}{suffix}{ext}"

    # Legend appearance/style options
    legend_figsize = style_dict.get('legend_figsize', (6, 2))
    legend_ncol = style_dict.get('legend_ncol', 1)
    legend_title = style_dict.get('legend_title', None)
    legend_title_fs = style_dict.get('legend_title_fontsize', None)
    legend_fs = style_dict.get('legend_fontsize', None)
    legend_loc = style_dict.get('legend_loc', 'center')
    legend_mode = style_dict.get('legend_mode', None)
    legend_frameon = style_dict.get('legend_frameon', True)
    legend_fill_figure = style_dict.get('legend_fill_figure', False)

    # Spacing/handle controls
    legend_handlelength = style_dict.get('legend_handlelength', None)
    legend_handleheight = style_dict.get('legend_handleheight', None)
    legend_labelspacing = style_dict.get('legend_labelspacing', None)
    legend_borderpad = style_dict.get('legend_borderpad', None)
    legend_markerscale = style_dict.get('legend_markerscale', None)
    legend_handletextpad = style_dict.get('legend_handletextpad', None)

    # Line widths
    legend_frame_linewidth = style_dict.get('legend_frame_linewidth', None)
    legend_handle_linewidth = style_dict.get('legend_handle_linewidth', None)

    legend_kw = dict(ncol=legend_ncol, loc=legend_loc)
    # frame and mode handled per-branch
    if legend_handlelength is not None:
        legend_kw['handlelength'] = legend_handlelength
    if legend_handleheight is not None:
        legend_kw['handleheight'] = legend_handleheight
    if legend_labelspacing is not None:
        legend_kw['labelspacing'] = legend_labelspacing
    if legend_borderpad is not None:
        legend_kw['borderpad'] = legend_borderpad
    if legend_markerscale is not None:
        legend_kw['markerscale'] = legend_markerscale
    if legend_handletextpad is not None:
        legend_kw['handletextpad'] = legend_handletextpad
    if legend_mode is not None:
        legend_kw['mode'] = legend_mode
    if legend_title is not None:
        legend_kw['title'] = legend_title

    # Create the legend figure
    fig_leg = plt.figure(figsize=legend_figsize, dpi=dpi, constrained_layout=True)

    if legend_fill_figure:
        # Fill the entire figure area with the legend
        ax_leg = fig_leg.add_axes([0, 0, 1, 1])
        ax_leg.axis('off')
        legend_kw['loc'] = legend_loc or 'center'
        legend_kw['mode'] = legend_mode or 'expand'
        legend_kw['frameon'] = style_dict.get('legend_frameon', False)  # default off when filling
        legend = ax_leg.legend(handles, labels, **legend_kw)
    else:
        ax_leg = fig_leg.add_subplot(111)
        ax_leg.axis('off')
        legend_kw['frameon'] = legend_frameon
        legend = ax_leg.legend(handles, labels, **legend_kw)

    # Apply font sizes
    if legend_title and legend_title_fs:
        legend.get_title().set_fontsize(legend_title_fs)
    if legend_fs:
        for txt in legend.get_texts():
            txt.set_fontsize(legend_fs)

    # Apply frame linewidth
    if legend_frame_linewidth is not None and legend.get_frame() is not None:
        legend.get_frame().set_linewidth(legend_frame_linewidth)

    # Apply handle linewidth to visible legend handles
    if legend_handle_linewidth is not None:
        for h in getattr(legend, 'legend_handles', []) or []:
            if hasattr(h, 'set_linewidth'):
                try:
                    h.set_linewidth(legend_handle_linewidth)
                except Exception:
                    pass

    fig_leg.savefig(save_path, dpi=dpi)
    return fig_leg

In [ ]:
def plot_openknot_score_boxplot(
    openknot_score_df, 
    method_list, 
    round_list, 
    style=None,
    limit_to_na_mpnn_puzzles=False
):
    """
    Plots a boxplot of target_openknot_score by method for specified rounds.
    Prints stats (count, median) for each method.
    Accepts a style dict for customizing plot appearance and a palette dict for 
    colors (style['palette']).
    """
    # Merge user style with defaults
    merged_style = DEFAULT_BOX_STYLE.copy()
    if style:
        merged_style.update(style)
    palette = merged_style.get('palette', None)

    # Filter dataframe for selected rounds
    selected_df = openknot_score_df[
        (openknot_score_df['round'].isin(round_list))
    ]
    
    if limit_to_na_mpnn_puzzles:
        na_mpnn_puzzles = selected_df[selected_df['method'] == 'NA-MPNN']["puzzle"].unique()
        selected_df = selected_df[selected_df['puzzle'].isin(na_mpnn_puzzles)]

    # Only plot methods present in the data
    order = [
        method for method in method_list 
        if method in selected_df['method'].unique()
    ]

    # Necessary for legend handle/text alignment.
    base_fs = matplotlib.rcParams['legend.fontsize']
    if style.get("legend_fontsize", None) is not None:
        matplotlib.rcParams['legend.fontsize'] = style["legend_fontsize"]

    # Create the boxplot
    fig, ax = plt.subplots(
        1, 
        1, 
        figsize=merged_style['figsize'], 
        dpi=merged_style['dpi'], 
        constrained_layout=True
    )

    # box/whisker/median props controlled by style
    box_lw = merged_style.get('box_linewidth', None)
    whisker_lw = merged_style.get('whisker_linewidth', box_lw if box_lw is not None else 1)
    cap_lw = merged_style.get('cap_linewidth', whisker_lw)
    median_lw = merged_style.get('median_linewidth', whisker_lw)
    median_color = merged_style.get('median_linecolor', None)

    # outlier (flier) marker size and edge width
    flier_ms = merged_style.get('flier_markersize', None)
    flier_lw = merged_style.get('flier_linewidth', None)

    boxprops = {'linewidth': box_lw} if box_lw is not None else None
    whiskerprops = {'linewidth': whisker_lw}
    capprops = {'linewidth': cap_lw}
    medianprops = {'linewidth': median_lw}
    if median_color is not None:
        medianprops['color'] = median_color
    flierprops = {}
    if flier_lw is not None:
        flierprops['markeredgewidth'] = flier_lw

    sns.boxplot(
        data=selected_df,
        x='method',
        y='target_openknot_score',
        order=order,
        ax=ax,
        palette=palette,
        hue='method',
        hue_order=order,
        legend=True,
        boxprops=boxprops,
        whiskerprops=whiskerprops,
        capprops=capprops,
        medianprops=medianprops,
        fliersize=flier_ms,
        flierprops=flierprops or None,
    )

    # Legend
    if merged_style.get('show_legend', False):
        # Rebuild legend with extended controls
        handles, labels = ax.get_legend_handles_labels()
        for lg in ax.figure.legends:
            try:
                lg.remove()
            except Exception:
                pass
        legend_kwargs = dict(
            loc=merged_style.get('legend_loc', 'center'),
            ncol=merged_style.get('legend_ncol', 1),
            title=merged_style.get('legend_title'),
            frameon=merged_style.get('legend_frameon', True),
        )
        if merged_style.get('legend_mode') is not None:
            legend_kwargs['mode'] = merged_style['legend_mode']
        if merged_style.get('legend_handlelength') is not None:
            legend_kwargs['handlelength'] = merged_style['legend_handlelength']
        if merged_style.get('legend_handleheight') is not None:
            legend_kwargs['handleheight'] = merged_style['legend_handleheight']
        if merged_style.get('legend_labelspacing') is not None:
            legend_kwargs['labelspacing'] = merged_style['legend_labelspacing']
        if merged_style.get('legend_borderpad') is not None:
            legend_kwargs['borderpad'] = merged_style['legend_borderpad']
        if merged_style.get('legend_markerscale') is not None:
            legend_kwargs['markerscale'] = merged_style['legend_markerscale']
        if merged_style.get('legend_handletextpad') is not None:
            legend_kwargs['handletextpad'] = merged_style['legend_handletextpad']

        legend = ax.legend(handles, labels, **legend_kwargs)

        # Style legend fonts
        if merged_style.get('legend_title_fontsize') is not None and legend.get_title():
            legend.get_title().set_fontsize(merged_style['legend_title_fontsize'])
        if merged_style.get('legend_fontsize') is not None:
            for text in legend.get_texts():
                text.set_fontsize(merged_style['legend_fontsize'])

        # Apply handle linewidth to visible legend handles
        legend_handle_linewidth = merged_style.get('legend_handle_linewidth', None)
        if legend_handle_linewidth is not None:
            for h in getattr(legend, 'legend_handles', []) or []:
                if hasattr(h, 'set_linewidth'):
                    try:
                        h.set_linewidth(legend_handle_linewidth)
                    except Exception:
                        pass

        # Save legend separately if requested
        if merged_style.get('save_legend_separately') and merged_style.get('save_name'):
            _save_legend_from_axes(ax, merged_style['save_name'], merged_style)
    else:
        lg = ax.get_legend()
        if lg is not None:
            try:
                lg.remove()
            except Exception:
                pass

    # Set background color
    ax.set_facecolor(merged_style['background_color'])
    fig.patch.set_facecolor(merged_style['background_color'])

    # Set title if requested
    if merged_style['show_title'] and merged_style['title']:
        ax.set_title(
            merged_style['title'], fontsize=merged_style['title_fontsize']
        )

    # Set y-axis limits if provided
    if merged_style['y_max'] is not None:
        ax.set_ylim(top=merged_style['y_max'])
    if merged_style['y_min'] is not None:
        ax.set_ylim(bottom=merged_style['y_min'])

    # Set axis labels
    if merged_style['show_axis_labels']:
        if merged_style['x_label']:
            ax.set_xlabel(
                merged_style['x_label'], 
                fontsize=merged_style['axis_title_fontsize']
            )
        else:
            ax.set_xlabel("")
        if merged_style['y_label']:
            ax.set_ylabel(
                merged_style['y_label'], 
                fontsize=merged_style['axis_title_fontsize']
            )
        else:
            ax.set_ylabel("")
    else:
        ax.set_xlabel("")
        ax.set_ylabel("")

    # Set tick label size and visibility
    if not merged_style['show_xticks']:
        ax.set_xticks([])
    else:
        ax.tick_params(axis='x', labelsize=merged_style['tick_labelsize'])
    if not merged_style['show_yticks']:
        ax.set_yticks([])
    else:
        ax.tick_params(axis='y', labelsize=merged_style['tick_labelsize'])

    # Rotate x-axis labels for readability
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')

    # set axis/spine and tick widths, and optionally hide top/right spines
    spine_lw = merged_style.get('spine_linewidth', None)
    tick_w = merged_style.get('tick_width', None)
    if spine_lw is not None:
        for spine in ax.spines.values():
            spine.set_linewidth(spine_lw)
    if tick_w is not None:
        ax.tick_params(width=tick_w)
    if merged_style.get('hide_top_right_spines', False):
        if 'top' in ax.spines:
            ax.spines['top'].set_visible(False)
        if 'right' in ax.spines:
            ax.spines['right'].set_visible(False)
    
    # Print stats for each method
    stats = (
        selected_df.groupby('method')['target_openknot_score']
        .agg(count='count', median='median')
        .reset_index()
    )
    print(stats)

    # Save the figure if requested
    if merged_style.get('save_name'):
        fig.savefig(
            merged_style['save_name'], 
            dpi=merged_style['dpi'], 
        )
    plt.show()

    matplotlib.rcParams['legend.fontsize'] = base_fs

def plot_puzzle_heatmap_and_scatter(
    openknot_score_df, 
    method_list, 
    plot_rounds, 
    style_heatmap=None, 
    style_scatter=None
):
    """
    For each puzzle in the specified rounds, plot a heatmap of reactivity and a 
    scatter of target_openknot_score by method. Accepts style dicts for both 
    subplots.
    """
    # Merge user style with defaults for both subplots
    merged_heatmap_style = DEFAULT_HEATMAP_STYLE.copy()
    if style_heatmap:
        merged_heatmap_style.update(style_heatmap)

    merged_scatter_style = DEFAULT_SCATTER_STYLE.copy()
    if style_scatter:
        merged_scatter_style.update(style_scatter)

    # Extract colormap and palette for the plots
    cmap = merged_heatmap_style.get('cmap', None)
    scatter_palette = merged_scatter_style.get('palette', None)

    # Filter dataframe for selected rounds and methods
    puzzle_df = openknot_score_df[
        (openknot_score_df['method'].isin(method_list)) &
        (openknot_score_df['round'].isin(plot_rounds))
    ]

    # Identify columns containing reactivity values (excluding error columns)
    data_columns = [
        col for col in puzzle_df.columns
        if col.startswith('reactivity_') and 'error' not in col
    ]

    # Necessary for legend handle/text alignment.
    base_fs = matplotlib.rcParams['legend.fontsize']
    if merged_scatter_style.get("legend_fontsize", None) is not None:
        matplotlib.rcParams['legend.fontsize'] = merged_scatter_style["legend_fontsize"]

    # Iterate over each unique puzzle id
    for puzzle_id in puzzle_df.puzzle.unique():
        # Subset data for the current puzzle_id
        puzzle_data = puzzle_df[puzzle_df.puzzle == puzzle_id].copy()

        # Only plot methods present in the data
        order = [
            method for method in method_list 
            if method in puzzle_data['method'].unique()
        ]

        # Skip puzzles where only one method is present
        if len(puzzle_data.method.unique()) <= 1:
            continue

        # Convert 'method' into an ordered categorical for consistent plotting
        puzzle_data['method_idx'] = puzzle_data['method'].map(
            lambda x: order.index(x) if x in order else -1
        )

        # Sort by 'method' (in method_order) then ascending 
        # 'target_openknot_score'
        puzzle_data = puzzle_data.sort_values(
            by=['method_idx', 'target_openknot_score'],
            ascending=[True, True]
        )

        # Assign a reverse index for plotting (so best is at the top)
        puzzle_data['sorted_inds'] = np.arange(len(puzzle_data))[::-1]

        # Pull out information from the top row, for indexing and labels
        top_row = puzzle_data.iloc[0]
        sequence = top_row['sequence']
        design_seq = top_row['design_seq']

        # Find the start/end indices of design_seq within sequence
        start_idx = sequence.find(design_seq)
        if start_idx == -1:
            raise ValueError(
                "'design_seq' not found in 'sequence'" + \
                f"for Puzzle ID {puzzle_id}"
            )
        end_idx = start_idx + len(design_seq)

        # Get the reference structure labels for positions corresponding to 
        # design_seq
        ref_structure = top_row['ref_structure']
        col_labels = list(ref_structure)[start_idx:end_idx]

        # Extract columns for those positions (reactivity values for the design 
        # region)
        features_subset = puzzle_data[data_columns].iloc[:, start_idx:end_idx]
        heatmap_data = features_subset

        # Create the figure with two subplots: heatmap (left), scatter (right)
        fig, (ax_heatmap, ax_scatter) = plt.subplots(
            nrows=1,
            ncols=2,
            figsize=merged_heatmap_style['figsize'],
            dpi=merged_heatmap_style['dpi'],
            gridspec_kw={'width_ratios': [3, 1]},
            constrained_layout=True
        )

        # --- Heatmap (left) ---
        hm = sns.heatmap(
            heatmap_data,
            cbar=True,
            yticklabels=False,
            xticklabels=col_labels,
            vmin=merged_heatmap_style['vmin'],
            vmax=merged_heatmap_style['vmax'],
            ax=ax_heatmap,
            cmap=cmap
        )

        cbar = hm.collections[0].colorbar
        cbar.ax.tick_params(
            labelsize = merged_heatmap_style.get('tick_labelsize', 10),
            width = merged_heatmap_style.get('tick_width', None)
        )

        for coll in hm.collections:
            if isinstance(coll, QuadMesh):
                coll.set_linewidth(0)
                coll.set_edgecolor('face')
                coll.set_antialiased(False)
                if merged_heatmap_style.get('rasterize_heatmap', True):
                    coll.set_rasterized(True)

        # Grouped y-tick labels, and rotation of y-tick labels.
        method_counts = puzzle_data.groupby('method')[
            'target_openknot_score'
        ].count()
        order_method_counts = method_counts[order]
        cumulative_counts = order_method_counts.cumsum()
        group_centers = cumulative_counts - order_method_counts / 2
        ax_heatmap.set_yticks(group_centers)
        ax_heatmap.set_yticklabels(order)
        for label in ax_heatmap.get_yticklabels():
            label.set_rotation(90)
            label.set_ha('center')
            label.set_va('center')

        # Set background color for heatmap and figure
        ax_heatmap.set_facecolor(merged_heatmap_style['background_color'])
        fig.patch.set_facecolor(merged_heatmap_style['background_color'])

        # Set title for heatmap
        if merged_heatmap_style['show_title'] and merged_heatmap_style['title']:
            ax_heatmap.set_title(
                merged_heatmap_style['title'], 
                fontsize=merged_heatmap_style['title_fontsize']
            )
        else:
            ax_heatmap.set_title("")

        # Set axis labels for heatmap
        if merged_heatmap_style['show_axis_labels']:
            if merged_heatmap_style['x_label']:
                ax_heatmap.set_xlabel(
                    merged_heatmap_style['x_label'], 
                    fontsize=merged_heatmap_style['axis_title_fontsize']
                )
            if merged_heatmap_style['y_label']:
                ax_heatmap.set_ylabel(
                    merged_heatmap_style['y_label'], 
                    fontsize=merged_heatmap_style['axis_title_fontsize']
                )
        else:
            ax_heatmap.set_xlabel("")
            ax_heatmap.set_ylabel("")

        # Set tick label size and visibility for heatmap
        if merged_heatmap_style['show_ticks']:
            ax_heatmap.tick_params(
                axis='both', 
                labelsize=merged_heatmap_style['tick_labelsize'],
                length=0
            )
        else:
            ax_heatmap.set_xticks([])
            ax_heatmap.set_yticks([])
        #plt.setp(ax_heatmap.get_xticklabels(), fontweight='bold')
        ax_heatmap.tick_params(axis='x', labelrotation=0)

        ax_heatmap.grid(False)

        # --- Scatter (right) ---
        sns.scatterplot(
            data=puzzle_data,
            x='target_openknot_score',
            y='sorted_inds',
            s=merged_scatter_style['s'],
            ax=ax_scatter,
            hue='method',
            palette=scatter_palette,
            hue_order=order,
        )

        # Set background color for scatter
        ax_scatter.set_facecolor(merged_scatter_style['background_color'])

        # Setup the legend.
        if merged_scatter_style.get('show_legend', False):
            handles, labels = ax_scatter.get_legend_handles_labels()
            lg_prev = ax_scatter.get_legend()
            if lg_prev:
                lg_prev.remove()

            legend_kwargs = dict(
                loc=merged_scatter_style.get('legend_loc', 'upper left'),
                ncol=merged_scatter_style.get('legend_ncol', 1),
                title=merged_scatter_style.get('legend_title'),
                frameon=merged_scatter_style.get('legend_frameon', True),
            )
            if merged_scatter_style.get('legend_mode') is not None:
                legend_kwargs['mode'] = merged_scatter_style['legend_mode']
            if merged_scatter_style.get('legend_handlelength') is not None:
                legend_kwargs['handlelength'] = merged_scatter_style['legend_handlelength']
            if merged_scatter_style.get('legend_handleheight') is not None:
                legend_kwargs['handleheight'] = merged_scatter_style['legend_handleheight']
            if merged_scatter_style.get('legend_labelspacing') is not None:
                legend_kwargs['labelspacing'] = merged_scatter_style['legend_labelspacing']
            if merged_scatter_style.get('legend_borderpad') is not None:
                legend_kwargs['borderpad'] = merged_scatter_style['legend_borderpad']
            if merged_scatter_style.get('legend_markerscale') is not None:
                legend_kwargs['markerscale'] = merged_scatter_style['legend_markerscale']
            if merged_scatter_style.get('legend_handletextpad') is not None:
                legend_kwargs['handletextpad'] = merged_scatter_style['legend_handletextpad']

            legend = ax_scatter.legend(handles=handles, labels=labels, **legend_kwargs)

            if merged_scatter_style.get('legend_title_fontsize') is not None and legend.get_title():
                legend.get_title().set_fontsize(merged_scatter_style['legend_title_fontsize'])
            if merged_scatter_style.get('legend_fontsize') is not None:
                for text in legend.get_texts():
                    text.set_fontsize(merged_scatter_style['legend_fontsize'])

            # Style legend handle linewidth
            legend_handle_linewidth = merged_scatter_style.get('legend_handle_linewidth', None)
            # Apply handle linewidth to visible legend handles
            if legend_handle_linewidth is not None:
                for h in getattr(legend, 'legend_handles', []) or []:
                    if hasattr(h, 'set_linewidth'):
                        try:
                            h.set_linewidth(legend_handle_linewidth)
                        except Exception:
                            pass
        else:
            lg = ax_scatter.get_legend()
            if lg:
                lg.remove()

        # Set the y-limit.
        ax_scatter.set_ylim((0-0.5, len(puzzle_data)-0.5))

        # Set title for scatter
        if merged_scatter_style['show_title'] and merged_scatter_style['title']:
            ax_scatter.set_title(
                merged_scatter_style['title'], 
                fontsize=merged_scatter_style['title_fontsize']
            )

        # Set axis labels for scatter
        if merged_scatter_style['show_axis_labels']:
            if merged_scatter_style['x_label']:
                ax_scatter.set_xlabel(
                    merged_scatter_style['x_label'], 
                    fontsize=merged_scatter_style['axis_title_fontsize']
                )
            if merged_scatter_style.get('y_label'):
                ax_scatter.set_ylabel(
                    merged_scatter_style['y_label'], 
                    fontsize=merged_scatter_style['axis_title_fontsize']
                )
            else:
                ax_scatter.set_ylabel("")
        else:
            ax_scatter.set_xlabel("")
            ax_scatter.set_ylabel("")

        # Set tick label size and visibility for scatter
        if not merged_scatter_style['show_xticks']:
            ax_scatter.set_xticks([])
        else:
            ax_scatter.tick_params(axis='x', labelsize=merged_scatter_style['tick_labelsize'])
        if not merged_scatter_style['show_yticks']:
            ax_scatter.set_yticks([])
        else:
            ax_scatter.tick_params(axis='y', labelsize=merged_scatter_style['tick_labelsize'])

        # apply spine/tick styling to heatmap & scatter axes as well
        spine_lw_h = merged_heatmap_style.get('spine_linewidth', merged_scatter_style.get('spine_linewidth', None))
        tick_w_h = merged_heatmap_style.get('tick_width', merged_scatter_style.get('tick_width', None))
        hide_tr = merged_heatmap_style.get('hide_top_right_spines', merged_scatter_style.get('hide_top_right_spines', False))

        if spine_lw_h is not None:
            for spine in ax_heatmap.spines.values():
                spine.set_linewidth(spine_lw_h)
            for spine in ax_scatter.spines.values():
                spine.set_linewidth(spine_lw_h)

        if tick_w_h is not None:
            ax_heatmap.tick_params(width=tick_w_h)
            ax_scatter.tick_params(width=tick_w_h)

        if hide_tr:
            if 'top' in ax_heatmap.spines:
                ax_heatmap.spines['top'].set_visible(False)
            if 'right' in ax_heatmap.spines:
                ax_heatmap.spines['right'].set_visible(False)
            if 'top' in ax_scatter.spines:
                ax_scatter.spines['top'].set_visible(False)
            if 'right' in ax_scatter.spines:
                ax_scatter.spines['right'].set_visible(False)

        # Identify row boundaries where 'method' changes and draw horizontal 
        # lines
        boundaries = []
        previous_method = puzzle_data['method'].iloc[0]
        for i in range(1, len(puzzle_data)):
            current_method = puzzle_data['method'].iloc[i]
            if current_method != previous_method:
                boundaries.append(i)
                previous_method = current_method
        for boundary in boundaries:
            ax_heatmap.axhline(boundary, color='silver', linewidth=1)
            ax_scatter.axhline(
                len(puzzle_data)-boundary-0.5, color='silver', linewidth=1
            )

        # Save the figure if requested
        if merged_heatmap_style.get('save_name'):
            # Save legend separately if requested
            if merged_scatter_style.get('show_legend', False) and merged_scatter_style.get('save_legend_separately', False):
                _save_legend_from_axes(ax_scatter, merged_heatmap_style['save_name'] % puzzle_id, merged_scatter_style)

            fig.savefig(
                merged_heatmap_style['save_name'] % puzzle_id, 
                dpi=fig.dpi
            )

        # Print stats for this puzzle (count and median by method)
        stats = (
            puzzle_data.groupby('method')['target_openknot_score']
            .agg(count='count', median='median')
            .reset_index()
        )
        print(f"Stats for puzzle {puzzle_id}:")
        print(stats)
        plt.show() 

    matplotlib.rcParams['legend.fontsize'] = base_fs

In [ ]:
# EPK-17_data.v2.0.0.csv
openknot_score_df = pd.read_csv(openknot_data_filepath)

openknot_score_df['design_seq'] = openknot_score_df.apply(
    lambda row: row['sequence'][row['sub_start'] - 1 : row['sub_end']],
    axis=1
)

reactivity_columns = [col for col in openknot_score_df.columns if ('reactivity' in col) and not ('error' in col)]

reactivity_vec_list = []
reactivity_vec_length_list = []
for i,r in openknot_score_df.iterrows():
    reactivity_vec = np.array(openknot_score_df.iloc[i][reactivity_columns].to_list()[openknot_score_df.iloc[i]['sub_start'] - 1 : openknot_score_df.iloc[i]['sub_end']])
    reactivity_vec_list.append(reactivity_vec)
    reactivity_vec_length_list.append(len(reactivity_vec))

openknot_score_df['reactivity_vec'] = reactivity_vec_list
openknot_score_df['reactivity_vec_length'] = reactivity_vec_length_list

openknot_score_df = openknot_score_df[
    (openknot_score_df['SN_filter'] == 1) &
    (openknot_score_df['method'].isin(method_list)) &
    (~openknot_score_df['id'].str.contains('_based_on_'))
]

openknot_score_df['method'] = openknot_score_df['method'].map(
    method_name_to_display_name
)
method_list = list(map(lambda x: method_name_to_display_name[x], method_list))

# Plot All Puzzle Metrics

In [ ]:
plot_openknot_score_boxplot(
    openknot_score_df, 
    method_list, 
    [1], 
    style=DEFAULT_BOX_STYLE | {
        'save_name': "/home/akubaney/projects/na_mpnn/figures/matplotlib/openknot_round_5.svg",
        'legend_ncol': 4,
        'legend_figsize': (64 / 25.4, 5 / 25.4)
    },
    limit_to_na_mpnn_puzzles=True
)

In [ ]:
plot_openknot_score_boxplot(
    openknot_score_df, 
    method_list, 
    [2], 
    style=DEFAULT_BOX_STYLE | {
        'save_name': "/home/akubaney/projects/na_mpnn/figures/matplotlib/openknot_round_6.svg",
        'figsize': (55 / 25.4, 40 / 25.4),
    },
    limit_to_na_mpnn_puzzles=True
)

In [ ]:
plot_openknot_score_boxplot(
    openknot_score_df, 
    method_list, 
    [3], 
    style=DEFAULT_BOX_STYLE | {
        'save_name': "/home/akubaney/projects/na_mpnn/figures/matplotlib/openknot_round_7a.svg",
        'legend_ncol': 2,
        'legend_figsize': (32 / 25.4, 5 / 25.4)
    },
    limit_to_na_mpnn_puzzles=True
)

In [ ]:
plot_openknot_score_boxplot(
    openknot_score_df, 
    method_list, 
    [4], 
    style=DEFAULT_BOX_STYLE | {
        'show_legend': True,
        'save_name': "/home/akubaney/projects/na_mpnn/figures/matplotlib/openknot_round_7b.svg",
        'legend_ncol': 2,
        'legend_figsize': (32 / 25.4, 5 / 25.4)
    },
    limit_to_na_mpnn_puzzles=True
)

# Plot Individual Puzzle Metrics

In [ ]:
plot_puzzle_heatmap_and_scatter(
    openknot_score_df, 
    ['WT','NA-MPNN','gRNAde', 'gRNAde2'], 
    [2],
    style_heatmap=DEFAULT_HEATMAP_STYLE | {
        'save_name': "/home/akubaney/projects/na_mpnn/figures/matplotlib/openknot_round_6_%s.svg"
    },
    style_scatter=DEFAULT_SCATTER_STYLE
)